<a href="https://colab.research.google.com/github/AlexandreSarmento/BooleanTumorGrowthKinetics/blob/main/iKM_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BINARIZATION THROUGH CLUSTERING METHODS NOTEBOOK
### Here we are adapting a previously developed approach called iterative k-means binarization with a clustering depth (d) of 1 (KM1) or 2 (KM2) or 3 (KM3). The number of cluster (k) must be greater than or equal the number of experimental measurement (n) k >= n. Just in case k = 2**d.

# INSTALL PACKAGES AND LIBRARIES

In [ ]:
!pip3 install biopython

In [ ]:
!pip install bitarray

In [ ]:
import math, numpy, sys
from bitarray import bitarray
import csv, array
from Bio import Cluster # import Biopython
import shutil

## Setting input data

In [ ]:
!mkdir morais
number_of_bin = 5
KMX = 3
inputFileName = '/content/popTimeSerie_'
outputFileName = '/content/morais/popTimeSerie_'
originalSeries = {}    # original series stored here as a list
binarySeries = {}    # structure same as the original but with binary series
inputoutput = {} # keys are combinations, with a list of compbinations that are outputs of it

mkdir: cannot create directory ‘morais’: File exists


# Function to binarize data through iterative k-means

In [ ]:
def decrementalKMeanBinarization(orig, clusters, reduction=0):
    clusterData = {}
    clusterRes = {}
    binSeries = {}
    if clusters < 2:
            return None
    while clusters >= 2:
            # 'NON' allows you to ignore specific species
            for k,v in list(orig.items()):
                    if k == "NON":
                            continue
                    data = []
                    for i in range(len(v)):
                            row = [i,v[i]]
                            data.append(row)
                    x = numpy.vstack(v)
                    clusterData[k] = x
                                        # Bio.Cluster.kcluster()
                    idx, error, nfound = Cluster.kcluster(x, nclusters=clusters, method='a', dist='e')
                    clusterRes[k] = (idx, error, nfound)

            # replace the values with the averages of the clusters
            for k,r in list(clusterRes.items()):
                    colors = []
                    idx, error, nfound = r
                    y = clusterData[k]
                    x = numpy.arange(len(y))
                    sizes = {}
                    sums = {}
                    for i in range(clusters):
                            sizes[i] = 0
                            sums[i] = 0
                    for i in range(len(idx)):
                            sizes[idx[i]] += 1
                            sums[idx[i]] += y[i]
                    # culculate sums
                    for i in list(sums.keys()):
                            sums[i] /= sizes[i]

                    # replace all the values with sums in the clusters
                    averaged = []
                    for i in idx:
                            averaged.append(sums[i][0])
                    orig[k] = averaged

            clusters //= 2
    # replace data with 0 and 1
    for k,v in list(orig.items()):
            if k == "NON":
                    continue
            mi = min(v)
            binVals = bitarray()
            for i in v:
                    if i == mi:
                            binVals.append(False)
                    else:
                            binVals.append(True)
            binSeries[k] = binVals

    return binSeries

In [ ]:
counter = 0
while (counter < number_of_bin):
    counter += 1
    print((str(counter) + " of " + str(number_of_bin) + "binarizations"))
    bMethod = KMX
    #Reduction methods
    reduction = 0
    csvfile = open(inputFileName +'.csv', 'r')
    r = csv.reader(csvfile)

    topLine = next(r)
    order = []
    # grabs each species,
    for t in topLine:
            # creates an empty list for each species
            originalSeries[t] = []
            order.append(t)
    for line in r:
            if line[0].startswith("#"):
                    continue
            words = line
            # fills the array for each species with the original data points
            for i in range(len(words)):
                    originalSeries[order[i]].append(float(words[i]))
    allConvergence = []
    allTimes = []
    # determines cluster depth (d). Number of clusters, k = 2^d
    if bMethod > 0:
        clusters = int(math.pow(2,bMethod))
        binarySeries = decrementalKMeanBinarization(dict(originalSeries), clusters, reduction)
    for i in binarySeries:
        sampleSize2= len(binarySeries[i])
    # write binarized data to file
    f = open(outputFileName + str(counter) + '.csv', "wb")
    for item in binarySeries:
        f.write(bytes(item,'utf-8'))
        values = str(binarySeries[item])
        values  = values.rstrip(')')
        values = values.rsplit('bitarray(')
        values = values[1]
        values = values.lstrip('\'')
        values = values.rstrip('\'')
        valueList = list(values)
        for sample in valueList:
            f.write(bytes("," + sample,'utf-8'))
            #f.write("," + str(sample))
        f.write(bytes("\n",'utf-8'))

    f.close()

# Tranform folder of results to zip

In [ ]:
shutil.make_archive('/content/morais', 'zip', '/content/morais')

In [ ]:
shutil.rmtree('/content/hepG2')

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
